<a href="https://colab.research.google.com/github/miladziekanowska/stock-market-forecast-project/blob/main/Technical_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing libraries and fixing problems with Talib ;)**

In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar xvzf ta-lib-0.4.0-src.tar.gz
import os
os.chdir('ta-lib')
!./configure --prefix=/usr
!make
!sudo make install
os.chdir('../')
!pip install TA-Lib

In [ ]:
pip install Yahoo_fin


In [3]:
import datetime as dt
import pandas as pd
import numpy as np
import yahoo_fin.stock_info as yh
import talib as ta
import matplotlib.pyplot as plt
import pandas_datareader as web
import matplotlib.dates as mdates
pd.set_option('display.max_columns', 20)

In [ ]:
ticker_list = yh.tickers_dow()
historical_datas = {}
for ticker in ticker_list:
    historical_datas[ticker] = yh.get_data(ticker, start_date="01/01/1993", end_date='now', index_as_date = True, interval="1d")

In [ ]:
historical_datas['AAPL']

**RSI**

In [34]:
historical_datas['AAPL']['RSI'] = ta.RSI(historical_datas['AAPL']['close'].tail(400))

In [ ]:
fig, axs = plt.subplots(2, 1, gridspec_kw={"height_ratios":[3,1]}, figsize=(10, 6))

axs[0].plot(historical_datas['AAPL']['close'].tail(400))
axs[1].axhline(y=70, color='r', linestyle='--')
axs[1].axhline(y=30, color='g', linestyle='--')
axs[1].plot(historical_datas['AAPL']['RSI'].tail(400), color='orange')

In [36]:
historical_datas['AAPL']['RSI_signal'] = 'HOLD'  # Inicjalizacja sygnałów na 'HOLD'

rsi_values = historical_datas['AAPL']['RSI']
rsi_signals = []

#Dopisanie sygnałów do kolumny RSI_signals
for rsi in rsi_values:
    if rsi < 30:
        rsi_signals.append('BUY')
    elif rsi > 70:
        rsi_signals.append('SELL')
    else:
        rsi_signals.append('HOLD')

historical_datas['AAPL']['RSI_signal'] = rsi_signals


In [ ]:
historical_datas['AAPL'].tail(30)

**MACD**

In [ ]:
historical_datas['AAPL']['MACD'], historical_datas['AAPL']['MACD_signalline'], macd_hist = ta.MACD(historical_datas['AAPL']['close'].tail(400))

fig, axs = plt.subplots(2, 1, gridspec_kw={"height_ratios":[3,1]}, figsize=(10, 6))

c = ["red" if cl < 0 else "green" for cl in macd_hist]

axs[0].plot(historical_datas['AAPL']['close'].tail(400))
axs[1].plot(historical_datas['AAPL']['MACD'], 'b-')
axs[1].plot(historical_datas['AAPL']['MACD_signalline'], 'b-', color='red')
axs[1].bar(macd_hist.index, macd_hist, color=c)

In [ ]:
historical_datas['AAPL'].tail(50)

In [40]:

signals = []
position = 'HOLD'

for i in range(len(historical_datas['AAPL'])):
    if i == 0:
        signals.append(position)
    else:
        if historical_datas['AAPL']['MACD'][i] > historical_datas['AAPL']['MACD_signalline'][i] and historical_datas['AAPL']['MACD'][i - 1] <= historical_datas['AAPL']['MACD_signalline'][i - 1]:
            position = 'BUY'
        elif historical_datas['AAPL']['MACD'][i] < historical_datas['AAPL']['MACD_signalline'][i] and historical_datas['AAPL']['MACD'][i - 1] >= historical_datas['AAPL']['MACD_signalline'][i - 1]:
            position = 'SELL'
        else:
            position = 'HOLD'

        signals.append(position)

historical_datas['AAPL']['MACD_signal'] = signals


**Williams %R**

In [41]:
historical_datas['AAPL']['WillR'] = ta.WILLR(historical_datas['AAPL']['high'], historical_datas['AAPL']['low'], historical_datas['AAPL']['close'])

In [ ]:
fig, axs = plt.subplots(2, 1, gridspec_kw={"height_ratios":[3,1]}, figsize=(10, 6))

axs[0].plot(historical_datas['AAPL']['close'].tail(200))
axs[1].axhline(y=-80, color='r', linestyle='--')
axs[1].axhline(y=-20, color='g', linestyle='--')
axs[1].plot(historical_datas['AAPL']['WillR'].tail(200), color='orange')

In [43]:
historical_datas['AAPL']['WillR_signal'] = 'HOLD'

willr_values = historical_datas['AAPL']['WillR']
willr_signals = []


for willr in willr_values:
    if willr < -80:
        willr_signals.append('BUY')
    elif willr > -20:
        willr_signals.append('SELL')
    else:
        willr_signals.append('HOLD')

historical_datas['AAPL']['WillR_signal'] = willr_signals

In [ ]:
historical_datas['AAPL'].tail(50)

**CCI**

In [45]:
historical_datas['AAPL']['CCI'] = ta.CCI(historical_datas['AAPL']['high'], historical_datas['AAPL']['low'], historical_datas['AAPL']['close'])

In [ ]:
fig, axs = plt.subplots(2, 1, gridspec_kw={"height_ratios":[3,1]}, figsize=(10, 6))

axs[0].plot(historical_datas['AAPL']['close'].tail(200))
axs[1].axhline(y=200, color='r', linestyle='--')
axs[1].axhline(y=-200, color='g', linestyle='--')
axs[1].plot(historical_datas['AAPL']['CCI'].tail(200), color='orange')

In [47]:
historical_datas['AAPL']['CCI_signals'] = 'HOLD'

cci_values = historical_datas['AAPL']['CCI']
cci_signals = []


for cci in cci_values:
    if cci > 200 :
        cci_signals.append('BUY')
    elif cci < -200:
        cci_signals.append('SELL')
    else:
        cci_signals.append('HOLD')

historical_datas['AAPL']['CCI_signals'] = cci_signals

**BOP**

In [48]:
historical_datas['AAPL']['BOP'] = ta.BOP(historical_datas['AAPL']['open'], historical_datas['AAPL']['high'], historical_datas['AAPL']['low'], historical_datas['AAPL']['close'])

In [ ]:
fig, axs = plt.subplots(2, 1, gridspec_kw={"height_ratios":[3,1]}, figsize=(10, 6))

axs[0].plot(historical_datas['AAPL']['close'].tail(200))
axs[1].axhline(y=0, color='g', linestyle='--')
axs[1].plot(historical_datas['AAPL']['BOP'].tail(200), color='orange')

In [50]:
historical_datas['AAPL']['BOP_signals'] = 'HOLD'

bop_values = historical_datas['AAPL']['BOP']
bop_signals = []


for bop in bop_values:
    if bop > 0 :
        bop_signals.append('BUY')
    elif bop < 0:
        bop_signals.append('SELL')
    else:
        bop_signals.append('HOLD')

historical_datas['AAPL']['BOP_signals'] = bop_signals

In [ ]:
historical_datas['AAPL'].tail(50)

In [52]:
def calculate_signals_for_symbol(symbol):
    historical_data = yh.get_data(symbol, start_date="01/01/1993", end_date='now', index_as_date=True, interval="1d")

    historical_data['RSI'] = ta.RSI(historical_data['close'].tail(400))

    historical_data['RSI_signal'] = 'HOLD'
    rsi_signals = []

    for rsi in historical_data['RSI']:
        if rsi < 30:
            rsi_signals.append('BUY')
        elif rsi > 70:
            rsi_signals.append('SELL')
        else:
            rsi_signals.append('HOLD')

    historical_data['RSI_signal'] = rsi_signals

    historical_data['MACD'], historical_data['MACD_signalline'], macd_hist = ta.MACD(historical_data['close'].tail(400))
    signals = []
    position = 'HOLD'

    for i in range(len(historical_data)):
        if i == 0:
            signals.append(position)
        else:
            if historical_data['MACD'][i] > historical_data['MACD_signalline'][i] and historical_data['MACD'][i - 1] <= historical_data['MACD_signalline'][i - 1]:
                position = 'BUY'
            elif historical_data['MACD'][i] < historical_data['MACD_signalline'][i] and historical_data['MACD'][i - 1] >= historical_data['MACD_signalline'][i - 1]:
                position = 'SELL'
            else:
                position = 'HOLD'

            signals.append(position)

    historical_data['MACD_signal'] = signals

    historical_data['WillR'] = ta.WILLR(historical_data['high'], historical_data['low'], historical_data['close'])
    historical_data['WillR_signal'] = 'HOLD'
    willr_signals = []

    for willr in historical_data['WillR']:
        if willr < -80:
            willr_signals.append('BUY')
        elif willr > -20:
            willr_signals.append('SELL')
        else:
            willr_signals.append('HOLD')

    historical_data['WillR_signal'] = willr_signals

    historical_data['CCI'] = ta.CCI(historical_data['high'], historical_data['low'], historical_data['close'])
    historical_data['CCI_signals'] = 'HOLD'
    cci_signals = []

    for cci in historical_data['CCI']:
        if cci > 200:
            cci_signals.append('BUY')
        elif cci < -200:
            cci_signals.append('SELL')
        else:
            cci_signals.append('HOLD')

    historical_data['CCI_signals'] = cci_signals

    historical_data['BOP'] = ta.BOP(historical_data['open'], historical_data['high'], historical_data['low'], historical_data['close'])
    historical_data['BOP_signals'] = 'HOLD'
    bop_signals = []

    for bop in historical_data['BOP']:
        if bop > 0:
            bop_signals.append('BUY')
        elif bop < 0:
            bop_signals.append('SELL')
        else:
            bop_signals.append('HOLD')

    historical_data['BOP_signals'] = bop_signals

    return historical_data

ticker_list = yh.tickers_dow()
historical_datas = {}

for ticker in ticker_list:
    historical_datas[ticker] = calculate_signals_for_symbol(ticker)

In [60]:
historical_datas['V']

,open,high,low,close,adjclose,volume,ticker,RSI,RSI_signal,MACD,MACD_signalline,MACD_signal,WillR,WillR_signal,CCI,CCI_signals,BOP,BOP_signals
2008-03-19,14.875000,17.250000,13.750000,14.125000,12.679029,708486000,V,NaN,HOLD,NaN,NaN,HOLD,NaN,HOLD,NaN,HOLD,-0.214286,SELL
2008-03-20,14.600000,16.250000,14.375000,16.087500,14.440630,198985200,V,NaN,HOLD,NaN,NaN,HOLD,NaN,HOLD,NaN,HOLD,0.793333,BUY
2008-03-24,16.809999,16.924999,14.750000,14.932500,13.403866,149566400,V,NaN,HOLD,NaN,NaN,HOLD,NaN,HOLD,NaN,HOLD,-0.863219,SELL
2008-03-25,15.145000,16.062500,14.955000,15.812500,14.193782,87092000,V,NaN,HOLD,NaN,NaN,HOLD,NaN,HOLD,NaN,HOLD,0.602708,BUY
2008-03-26,15.682500,16.120001,15.392500,15.990000,14.353113,43111600,V,NaN,HOLD,NaN,NaN,HOLD,NaN,HOLD,NaN,HOLD,0.422680,BUY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-15,242.009995,242.669998,239.350006,239.830002,239.830002,3770100,V,53.038524,HOLD,1.264748,1.336310,SELL,-37.488615,HOLD,61.263458,HOLD,-0.656626,SELL
2023-08-16,236.529999,241.070007,236.020004,239.289993,239.289993,3218600,V,51.743521,HOLD,1.123287,1.293705,HOLD,-45.507892,HOLD,-23.474879,HOLD,0.546533,BUY
2023-08-17,240.000000,240.380005,236.490005,237.369995,237.369995,4177600,V,47.319622,HOLD,0.846492,1.204263,HOLD,-75.632231,HOLD,-79.557688,HOLD,-0.676094,SELL
2023-08-18,235.580002,239.050003,235.300003,238.169998,238.169998,4344800,V,49.265986,HOLD,0.683802,1.100171,HOLD,-66.436791,HOLD,-112.953521,HOLD,0.690666,BUY
